# ComfyUI Node Installer

This notebook installs ComfyUI nodes for SwarmUI integration. It will:
- Check for ComfyUI directory structure
- Install or update specified ComfyUI nodes
- Handle requirements installation for each node

## Configuration

Set up the ComfyUI directory path and list of nodes to install:

In [ ]:
import os
import subprocess
import sys
from pathlib import Path

# Set ComfyUI directory to match SwarmUI's structure
COMFYUI_DIR = "./dlbackend/ComfyUI"

# List of nodes to install
NODES = [
    "https://github.com/Comfy-Org/ComfyUI-Manager",
    "https://github.com/kijai/ComfyUI-KJNodes",
    "https://github.com/aria1th/ComfyUI-LogicUtils",
    "https://github.com/crystian/ComfyUI-Crystools",
    "https://github.com/Kosinkadink/ComfyUI-VideoHelperSuite",
    "https://github.com/rgthree/rgthree-comfy",
    "https://github.com/calcuis/gguf",
    "https://github.com/city96/ComfyUI-GGUF"
]

print(f"ComfyUI Directory: {COMFYUI_DIR}")
print(f"Number of nodes to install: {len(NODES)}")

## Directory Validation

Check if the ComfyUI directory exists:

In [ ]:
# Check if ComfyUI directory exists
comfyui_path = Path(COMFYUI_DIR)

if not comfyui_path.exists():
    print(f"❌ Error: ComfyUI directory not found at {COMFYUI_DIR}")
    print("Please run SwarmUI installation first")
    raise FileNotFoundError(f"ComfyUI directory not found: {COMFYUI_DIR}")
else:
    print(f"✅ ComfyUI directory found at {COMFYUI_DIR}")
    
# Create custom_nodes directory if it doesn't exist
custom_nodes_dir = comfyui_path / "custom_nodes"
custom_nodes_dir.mkdir(exist_ok=True)
print(f"✅ Custom nodes directory ready at {custom_nodes_dir}")

## Virtual Environment Setup

Check for and activate ComfyUI's virtual environment if it exists:

In [ ]:
# Check for virtual environment
venv_path = comfyui_path / "venv" / "bin" / "activate"
swarm_no_venv = os.environ.get("SWARM_NO_VENV")

if venv_path.exists() and not swarm_no_venv:
    print("✅ ComfyUI virtual environment found")
    print("Note: In Jupyter, the virtual environment activation is handled differently.")
    print("Make sure you're running this notebook in the correct Python environment.")
else:
    print("ℹ️  No virtual environment found or SWARM_NO_VENV is set")
    print("Using system Python environment")

## Node Installation Functions

Define helper functions for installing nodes:

In [ ]:
def run_command(command, cwd=None):
    """Run a shell command and return the result"""
    try:
        result = subprocess.run(
            command, 
            shell=True, 
            cwd=cwd, 
            capture_output=True, 
            text=True, 
            check=True
        )
        return result.stdout
    except subprocess.CalledProcessError as e:
        print(f"❌ Command failed: {command}")
        print(f"Error: {e.stderr}")
        return None

def install_requirements(requirements_path):
    """Install requirements from requirements.txt file"""
    if requirements_path.exists():
        print(f"  📦 Installing requirements from {requirements_path}")
        command = f"{sys.executable} -m pip install --no-cache-dir -r {requirements_path}"
        result = run_command(command)
        if result is not None:
            print("  ✅ Requirements installed successfully")
        return result is not None
    return True

print("✅ Helper functions defined")

## Install/Update Nodes

Install or update each ComfyUI node:

In [ ]:
def install_nodes():
    """Install or update all ComfyUI nodes"""
    print("🚀 Installing ComfyUI nodes...\n")
    
    success_count = 0
    total_count = len(NODES)
    
    for i, repo in enumerate(NODES, 1):
        # Extract directory name from repo URL
        dir_name = repo.split("/")[-1]
        node_path = custom_nodes_dir / dir_name
        requirements_path = node_path / "requirements.txt"
        
        print(f"[{i}/{total_count}] Processing: {dir_name}")
        
        if node_path.exists():
            print(f"  📁 Node already exists - Updating...")
            # Update existing repository
            result = run_command("git pull", cwd=node_path)
            if result is not None:
                print(f"  ✅ Updated successfully")
                if install_requirements(requirements_path):
                    success_count += 1
            else:
                print(f"  ❌ Failed to update")
        else:
            print(f"  📥 Installing new node...")
            # Clone new repository
            command = f"git clone {repo} {node_path} --recursive"
            result = run_command(command)
            if result is not None:
                print(f"  ✅ Cloned successfully")
                if install_requirements(requirements_path):
                    success_count += 1
            else:
                print(f"  ❌ Failed to clone")
        
        print()  # Empty line for readability
    
    print(f"🎉 Node installation complete!")
    print(f"✅ Successfully processed: {success_count}/{total_count} nodes")
    
    if success_count < total_count:
        print(f"⚠️  {total_count - success_count} nodes had issues - check the output above")

# Run the installation
install_nodes()

## Verification

Verify that all nodes were installed correctly:

In [ ]:
# List installed nodes
print("📋 Installed ComfyUI Nodes:")
print("=" * 40)

installed_nodes = []
for repo in NODES:
    dir_name = repo.split("/")[-1]
    node_path = custom_nodes_dir / dir_name
    
    if node_path.exists():
        # Check if it's a git repository
        git_dir = node_path / ".git"
        status = "✅ Installed" if git_dir.exists() else "⚠️  Directory exists (not git)"
        installed_nodes.append(dir_name)
    else:
        status = "❌ Not found"
    
    print(f"{status} - {dir_name}")

print(f"\n📊 Summary: {len(installed_nodes)}/{len(NODES)} nodes installed")

## Next Steps

After running this notebook:

1. **Restart ComfyUI**: If ComfyUI was running, restart it to load the new nodes
2. **Check Node Manager**: Use ComfyUI-Manager to verify nodes are loaded correctly
3. **Test Functionality**: Try using the new nodes in your ComfyUI workflows
4. **Update Regularly**: Re-run this notebook periodically to update nodes

## Troubleshooting

If you encounter issues:
- Ensure you have git installed and accessible
- Check that you have proper permissions to write to the ComfyUI directory
- Verify your internet connection for downloading repositories
- Check the ComfyUI logs for any node loading errors